In [0]:
!pip install tensorflow==1.2.0 --ignore-installed

In [0]:
!pip install tflearn

In [0]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [0]:
import numpy as np
import tensorflow as tf
import tflearn
import random
import json

In [0]:
with open('data/intents.json') as file:
  data = json.load(file)

In [0]:
nltk.download('punkt')

In [0]:
words = []
labels = []
docs = []

for intent in data['intents']:
  for pattern in intent['patterns']:
    docs.append(pattern)
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds)
  if intent['tag'] not in labels:
    labels.append(intent['tag'])

In [11]:
print(words)
print(labels)
print(docs)

['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'ki', 'khobor', 'bhai', 'acho', 'cya', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'Leaving', 'Have', 'a', 'Good', 'day', 'how', 'old', 'how', 'old', 'is', 'tim', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', '?', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', '?', 'Id', 'like', 'to', 'buy', 'something', 'whats', 'on', 'the', 'menu', 'what', 'do', 'you', 'reccommend', '?', 'could', 'i', 'get', 'something', 'to', 'eat', 'when', 'are', 'you', 'guys', 'open', 'what', 'are', 'your', 'hours', 'hours', 'of', 'operation']
['greeting', 'goodbye', 'age', 'name', 'shop', 'hours']
['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up', 'ki khobor', 'bhai acho', 'cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day', 'how old', 'how old is tim', 'what is your age', 'how old are you', 'age?', 'what is your 